In [1]:
import os
import getpass

# Prompt the user for the API key
api_key = getpass.getpass("Enter your LANGSMITH API Key: ")

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = api_key

# You can now use these environment variables in your code
print("LANGSMITH_TRACING:", os.getenv("LANGSMITH_TRACING"))
print("LANGSMITH_API_KEY has been set.")

Enter your LANGSMITH API Key: ··········
LANGSMITH_TRACING: true
LANGSMITH_API_KEY has been set.


In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [3]:
!pip install langsmith

In [4]:
!pip install openai langchain_community

## Prompt 1

In [26]:
from langsmith import traceable, wrappers
from openai import Client

openai = wrappers.wrap_openai(Client())

MODEL = "gpt-4o-mini"

from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=MODEL,
    max_tokens=200
)

@traceable
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>
    """
    messages = [{'role': 'user', 'content': user_content}]
    return messages

In [27]:
from langsmith import Client

client = Client()
dataset_name = "Sentiment Dataset"


In [28]:
from langsmith.schemas import Example, Run

def correct_label(root_run: Run, example: Example) -> dict:
    score = root_run.outputs.get("output") == example.outputs.get("golden_answer")
    return {"score": int(score), "key": "correct_label"}

In [ ]:
from langsmith.evaluation import evaluate


results = evaluate(
    lambda inputs: get_sentiment(build_input_prompt(inputs["input"])),
    data=dataset_name,
    evaluators=[correct_label],
    experiment_prefix="Sentiment",
)

## Prompt 2

In [5]:
from langsmith import traceable, wrappers
from openai import Client

openai = wrappers.wrap_openai(Client())

MODEL = "gpt-4o-mini"

from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=MODEL,
    max_tokens=200
)

@traceable(name="prompt 2 refine")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

<ipython-input-5-1d3d92a0c7a8>:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
from langsmith import Client

client = Client()
dataset_name = "Sentiment Dataset"


In [7]:
from langsmith.schemas import Example, Run

def correct_label(root_run: Run, example: Example) -> dict:
    score = root_run.outputs.get("output") == example.outputs.get("golden_answer")
    return {"score": int(score), "key": "correct_label"}

In [8]:
from langsmith.evaluation import evaluate


results = evaluate(
    lambda inputs: get_sentiment(build_input_prompt(inputs["input"])),
    data=dataset_name,
    evaluators=[correct_label],
    experiment_prefix="Sentiment",
)

View the evaluation results for experiment: 'Sentiment-78e9c375' at:
https://smith.langchain.com/o/9c75e871-1666-51ff-aaa8-7c3e38165404/datasets/71866eb4-2615-43be-80da-05678ed760ec/compare?selectedSessions=e163bbb6-6a38-41e8-8600-5d3cca82cc0b




0it [00:00, ?it/s]